In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./data/events.csv')
df = df.dropna(subset=['itemid', 'transactionid'])
df['transactionid'] = df['transactionid'].astype(int)

In [3]:
df.sample(10)

,timestamp,visitorid,event,itemid,transactionid
1785812,1431961631573,407882,transaction,420238,9230
2314591,1436726535140,1297062,transaction,361545,8866
2531913,1437616618559,1376763,transaction,73311,16900
2440889,1437243633771,371606,transaction,175749,4829
2201410,1436286088769,581152,transaction,307733,17615
1694990,1431621613832,761482,transaction,438605,765
839637,1439497165479,933821,transaction,323506,5000
100157,1433556183373,1398811,transaction,266553,5579
2364899,1436923381205,728219,transaction,215868,1868
2574662,1437749398334,1293259,transaction,158123,14504


In [20]:
def cartesian(x, y):
    return np.transpose([np.tile(x, len(y)), np.repeat(y, len(x))])

In [39]:
def build_items_pairs(items):
    product = cartesian(items.values, items.values)
    return list(product)

In [37]:
aggregated = df.groupby(['transactionid'])['itemid'].agg(build_items_pairs)